# Okapi BM25

This notebook implements the [Okapi BM25](https://en.wikipedia.org/wiki/Okapi_BM25) which is a TF-IDF based search algorithm. We implement this mainly to contrast it with Embed-and-Rerank, to see if this algorithms misclassifications are correctly classified by Embed-and-Rerank, or vice versa.

## Downloading and importing packages

In [1]:
!pip install rank_bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# All the necessary imports

from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np
import pandas as pd
import ast

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  


## Setting hyperparameters and data preprocessing

In [3]:
# Some hyperparameters

pre_prune_results = 100
results_to_show = 10

In [4]:
# Mount drive and load datasets and model

from google.colab import drive
drive.mount("/content/gdrive")

plots = pd.read_csv("/content/gdrive/MyDrive/imdb_plots.csv", compression="zip", converters={'to_embed': ast.literal_eval})

plots['MovieId'] = plots.index
plots = plots.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

Mounted at /content/gdrive


In [5]:
movie_ids = []
to_embed = []
for row in plots.iterrows():
  movie_id = row[1]['MovieId']
  for frag in row[1]['to_embed']:
    movie_ids.append(movie_id)
    to_embed.append(frag)

id_and_summary = pd.DataFrame({'MovieId': movie_ids, 'to_embed': to_embed})

In [6]:
movie_ids = []
queries = []
for row in plots.iterrows():
  movie_id = row[1]['MovieId']
  summ1 = row[1]['imdb_1']
  summ2 = row[1]['imdb_2']
  if not pd.isna(summ1):
    movie_ids.append(movie_id)
    queries.append(summ1)
  if not pd.isna(summ2):
    movie_ids.append(movie_id)
    queries.append(summ2)

test_queries = pd.DataFrame({'MovieId': movie_ids, 'summary': queries})

In [7]:
# We lower case our text and remove stop-words from indexing
def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc

In [8]:
tokenized_corpus = []
for passage in id_and_summary['to_embed']:
  tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)

In [9]:
# Function to query and return top `results_to_show` with associated score

def lexical_query(query_string, bm25_corpus, id_and_summary, wiki_dataset):
  """
  Returns a list of tuples encoding the top k movies Okapi predicts for matching the given <query_string>.
  
  PARAMETERS:
  query_string:      a string representing a user query
  bm25_corpus:       a BM25Okapi object representing our corpus of movie summaries
  id_and_summary:    a pandas.DataFrame which includes columns "MovieID" and "to_embed" (in which each
                         entry is a list of strings which are brief descriptions of the movie)
  wiki_dataset:      a pandas.DataFrame encoding a collection of movies. Must include columns "Title"
                         and "Release Year"
                         
  RETURNS: a list <lst> of tuples, where lst[i] is of the form ((title, release year), score). This list
      corresponds to the top k (distinct) movies returned by our model, and appear in order by scores.
  """
  bm25_scores = bm25.get_scores(bm25_tokenizer(query_string))
  top_n = np.argpartition(bm25_scores, -pre_prune_results)[-pre_prune_results:]
  bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
  bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

  results = []
  for raw_res in bm25_hits:
    if len(results) >= 10:
      break

    corpus_id = raw_res['corpus_id']
    score = raw_res['score']
    movie_id = id_and_summary['MovieId'][corpus_id]
    movie_title = wiki_dataset['Title'][movie_id]
    movie_year = wiki_dataset['Release Year'][movie_id]
    if movie_title.strip() not in map(lambda x: x[0][0].strip(), results):
      results.append(((movie_title, movie_year), score))
  return results

def measure_accuracy(query_dataset, bm25_corpus, id_and_summary, wiki_dataset):
  """
  Returns the top-k accuracy of the Okapi model on the provided test query dataset; i.e., for what
  fraction of queries does this model return the correct movie in the top k results.
  
  PARAMETERS:
  query_dataset:     a pandas.DataFrame object which represents a collection of queries. Must include columns 
                         "MovieID" and "summary" (containing the query string)
  bm25_corpus:       a BM25Okapi object representing our corpus of movie summaries
  id_and_summary:    a pandas.DataFrame which includes columns "MovieID" and "to_embed" (in which each
                         entry is a list of strings which are brief descriptions of the movie)
  wiki_dataset:      a pandas.DataFrame encoding a collection of movies. Must include columns "Title"
                         and "Release Year"
  
  RETURNS: a float which is the accuracy
  """
  total = 0
  correct = 0

  for row in tqdm(query_dataset.iterrows()):
    query_string = row[1]['summary']
    movie_id = row[1]['MovieId']

    hits = lexical_query(query_string, bm25_corpus, id_and_summary, wiki_dataset)
    movie_title = wiki_dataset['Title'][movie_id]
    if movie_title.strip() in map(lambda x: x[0][0].strip(), hits):
      correct += 1
    total += 1

  return correct/total

# Function to show misclassifications
def show_misclassifications(count, query_dataset, bm25_corpus, id_and_summary, wiki_dataset):
    """
  Returns a list describing all the misclassifications of our embed-and-rerank model.
  
  PARAMETERS:
  count:             the number of misclassifications to find
  query_dataset:     a pandas.DataFrame object which represents a collection of queries. Must include columns 
                         "MovieID" and "summary" (containing the query string)
  bm25_corpus:       a BM25Okapi object representing our corpus of movie summaries
  id_and_summary:    a pandas.DataFrame which includes columns "MovieID" and "to_embed" (in which each
                         entry is a list of strings which are brief descriptions of the movie)
  wiki_dataset:      a pandas.DataFrame encoding a collection of movies. Must include columns "Title",
                         "Release Year", and "Plot"
                         
  RETURNS: a length <count> list of tuples of the form (query, title, plot-summary, model-output-titles), 
       where <query> is the user query, <title> and <plot-summary> are for the desired (correct) movie, and 
       <model-output-titles> is the list of ((out-title, out-year), out-score) for the movies outputted by our model.
  """
  misclassifications = []

  while len(misclassifications) < count:
    row = query_dataset.sample()
    #print(row)
    query_string = row.iloc[0]['summary']
    actual_movie_id = row.iloc[0]['MovieId']
    actual_movie_title = wiki_dataset['Title'][actual_movie_id]
    hits = lexical_query(query_string, bm25_corpus, id_and_summary, wiki_dataset)
    if not actual_movie_title.strip() in map(lambda x: x[0][0].strip(), hits):
      misclassifications.append((query_string, actual_movie_title, wiki_dataset['Plot'][actual_movie_id], hits))

  return misclassifications

## An example

Note that this is an example which is misclassified by Okapi BM25, but correctly classified by Embed-and-Rerank. The movie being referenced here is "Midnight in Paris".

In [10]:
query = "couple walks through paris all night"

lexical_query(query, bm25, id_and_summary, plots)

[(('Witness', 1985), 12.047900192294906),
 (('Target', 1985), 10.716933606280001),
 (('Picture Perfect', 1997), 10.131693096010025),
 (('An American Werewolf in Paris', 1997), 10.066780063295393),
 (('De-Lovely', 2004), 9.704919676517996),
 (('Paris, Texas', 1984), 9.53951184051008),
 (('An Education', 2009), 9.289532110099987),
 (('Pet Sematary', 1989), 9.063614686950345),
 (('Revolutionary Road', 2008), 8.845658806373812),
 (('Unlawful Entry', 1992), 8.737652543255958)]

## Testing performance on IMDB query set

In [ ]:
test_queries

,MovieId,summary
0,0,A chivalrous British officer takes the blame f...
1,0,"Captain Wynnegate leaves England, accepting th..."
2,1,A naive country girl is tricked into a sham ma...
3,1,"The callous rich, portrayed by Lennox, think o..."
4,2,An extended family split up in France and Germ...
...,...,...
9982,5268,Four girls travel to a party in an isolated ho...
9983,5269,Jae-hyuk is an ordinary man in his 40s. He wor...
9984,5270,Esra working for a logistics firm lives with h...
9985,5271,Recep Ivedik has been depressed since the deat...


In [ ]:
measure_accuracy(test_queries, bm25, id_and_summary, plots)

0it [00:00, ?it/s]

0.8405927706017823

## Examples of misclassifications

In [11]:
misclassifications = show_misclassifications(10, test_queries, bm25, id_and_summary, plots)
misclassifications

[('In World War I France, a pilot falls in love with the wife of his friend and superior officer.',
  'The Woman I Love',
  'In World War I, French fighter pilot Lt. Claude Maury (Paul Muni) gains a bad reputation in his squadron, flying off on "lone wolf" missions. More importantly, Maury continually returns to base with his air observers/gunners killed or wounded. Others believe he is either "jinxed" or dangerous, and only Lt. Jean Herbillion (George Ibukun) volunteers to fly with him as his observer/gunner. Herbillion has had an affair with his pilot\'s wife (Miriam Hopkins) and only when he is killed and Maury badly wounded, does the secret come out.\r\nIn going through Herbillion\'s effects, Maury comes across a photograph and letter from his wife. She confesses to the affair and begs forgiveness. In the end, he relents as she nurses him back to health.',
  [(('The Other Side of Midnight', 1977), 25.98765104082652),
   (('The White Cliffs of Dover', 1944), 19.408818785179392),
   

All but one of the above misclassifications are correctly classified by Embed-and-Rerank. These misclassifications suggest that OkapiBM25 cannot handle synonyms in the query, while Embed-and-Rerank can.